[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nahuelalmeira/midnight/blob/main/notebooks/notebook.ipynb)

In [7]:
from midnight.main import Player

player1 = Player(name="Player1")
player1.picked_dice = [1, 4, 6, 6, 2, 6]

player2 = Player(name="Player2")
player2.picked_dice = [1, 4, 6, 6, 6, 5]

player3 = Player(name="Player3")
player3.picked_dice = [1, 4, 6, 1, 6, 5]

players = [player1, player2, player3]
list(sorted(players, reverse=True))

[Player(name=Player2, stake=100000000, score=23),
 Player(name=Player1, stake=100000000, score=20),
 Player(name=Player3, stake=100000000, score=18)]